In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
train = pd.read_excel('./Participants_Data_News_category/Data_Train.xlsx')
test = pd.read_excel('./Participants_Data_News_category/Data_Test.xlsx')

In [3]:
# Importing the Libraries

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

# Download the Following Modules once

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /home/imaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/imaya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
print(train.shape)
train.head()

(7628, 2)


,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [5]:
print(test.shape)
test.head()

(2748, 1)


,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [6]:
train.drop_duplicates(inplace=True)

#A punctuations string for reference (added other valid characters from the dataset)

all_punctuations = string.punctuation + '‘’,:”][],'

#Method to remove punctuation marks from the data

def punc_remove(raw_text):
    no_punc = "".join([punc for punc in raw_text if punc not in all_punctuations])
    return no_punc

def stopword_remover(raw_text):
    words = raw_text.split()
    raw_text = " ".join([i for i in words if i not in stopwords.words('english')])
    return raw_text

lemmer = nltk.stem.WordNetLemmatizer()

def lem(words):
    return " ".join([lemmer.lemmatize(word,'v') for word in words.split()])


# All together 

def text_cleaner(raw):
    cleaned_text = stopword_remover(punc_remove(raw))
    return lem(cleaned_text)


In [7]:
train['CLEAN_STORY'] = train['STORY'].apply(text_cleaner)


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Creating a bag of words Dictionery of words from the Data

bow_dictionery = CountVectorizer().fit(train['CLEAN_STORY'])

len(bow_dictionery.vocabulary_)

bow = bow_dictionery.transform(train['CLEAN_STORY'])

print(bow.shape)


from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer().fit(bow)

storytfidf = tfidf.transform(bow)


(7551, 35189)


In [14]:
X_train, X_test, y_train, y_test = train_test_split( storytfidf, train["SECTION"], random_state = 21)

In [15]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train,y_train)

In [16]:
clf.score(X_test, y_test)

0.9221398305084746

In [11]:
#Importing the Pipeline module from sklearn
from sklearn.pipeline import Pipeline

#Initializing the pipeline with necessary transformations and the required classifier
pipe = Pipeline([('Bow', CountVectorizer()),
                ('TfIdf', TfidfTransformer()),
                ('Classifier',MultinomialNB())])


#Fitting the training data to the pipeline
pipe.fit(X_train,y_train)



In [ ]:
#Predicting the SECTION 
test_pred = pipe.predict(test['CLEAN_STORY'])

#Writing the predictions to an excel sheet
pd.DataFrame(test_pred, columns = ['SECTION']).to_excel('predictions.xlsx', index=False)
